In [1]:
import numpy as np
import matplotlib.pyplot as plt
from astropy.io.fits import getdata

import pandas as pd
numaster = getdata('numaster.fits')

from astropy.time import Time

cutoff_time = Time('2025-10-01T00:00:00').mjd
start_time = Time('2012-12-31T00:00:00').mjd

In [2]:
df = pd.read_csv('stray_light_det_v4a.txt', sep='\s+', names=['SEQID', 'FPM', 'SRC', 'DET0', 'DET1', 'DET2', 'DET3'])

# Filter out cases where the source is really bright:
bright_lim = 100.
df = df[df['SRC']<100.]


df['DET0123'] = df['DET0'] + df['DET1'] + df['DET2'] + df['DET3']
df['DET01'] = df['DET0'] + df['DET1']
df['DET12'] = df['DET1'] + df['DET2']
df['DET23'] = df['DET2'] + df['DET3']
df['DET03'] = df['DET0'] + df['DET3']

names = []
exp = []
ra = []
dec = []
pa = []
mjds = []
for row in df['SEQID']:
    nuind = np.where(numaster['OBSID'] == f'{row}')

    if (len(nuind[0]) == 0):
        names = np.append(names, '')
        exp = np.append(exp, 0)
        ra = np.append(ra, 0.)
        dec = np.append(dec, 0.)
        pa = np.append(pa, 0.)
        mjds = np.append(mjds, 0.)
    else:
        names = np.append(names, numaster['NAME'][nuind[0]].strip())
        exp = np.append(exp, numaster['EXPOSURE_A'][nuind[0]])
        ra = np.append(ra,numaster['RA'][nuind[0]] )
        dec = np.append(dec,numaster['DEC'][nuind[0]] )
        pa = np.append(pa,numaster['ROLL_ANGLE'][nuind[0]] )
        mjds = np.append(mjds,numaster['TIME'][nuind[0]] )


        
df['NAME'] = names
df['EXPOSURE'] = exp
df['RA'] = ra
df['DEC'] = dec
df['PA'] = pa
df['TIME'] = mjds

# Get rid of stuff with low exposure
df = df[(df['EXPOSURE']>1e3) & (df['TIME']<cutoff_time)& (df['TIME']>start_time)]


# Get rid of stuff near SgrA*, GalSurveys, Gal_Cen, or the GC Magnetar:
# Also Coma and Bullet cluster stuff
bad_name = ['Sgr', 'GalSurvey', 'Gal_Cen', 'gcmag', 'Coma', 'bullet', 'Bullet']
for bn in bad_name:   
    df = df[~(df['NAME'].str.startswith(bn))]

In [3]:
sig_list = []

merged_df = pd.read_csv('straycats3_merged.csv')



sig_list = merged_df['SEQID'].values

# with open('straycats2_merged.csv', 'r') as f:
#     for line in f:
#         fields = line.split(',')
# #         if fields[0].strip() == '':
# #             break
#         if fields[1] not in sig_list:
#             sig_list = np.append(sig_list, fields[1])
# sig_list = sig_list[1:]

In [4]:
print(len(df))
print(len(sig_list))

for seq in sig_list:
    try:
        seqid = float(seq)
    except:
        continue
    df = df[~(df['SEQID'] == seqid)].copy().reset_index(drop=True)
print(len(df))


614
123
547


In [5]:
# Filter out bright sources and short exposures
df = df[(df['SRC'] < 2.) & (df['EXPOSURE'] > 10e3)].copy().reset_index(drop=True)
print(len(df))

479


In [6]:
dfa = df[df['FPM']=='A'].copy().reset_index(drop=True)
dfb = df[df['FPM']=='B'].copy().reset_index(drop=True)

running_list = {'A':[], 'B':[]}


# Iteration step: find stuff 3-sigma

sig_limit = 5.0
for det_comb in ['DET0123', 'DET12', 'DET23', 'DET01', 'DET03', 'DET1', 'DET2', 'DET3', 'DET0']:
    print(det_comb)
    print()
    
    use = {'A':np.full(len(dfa), True, dtype=bool),
           'B':np.full(len(dfb), True, dtype=bool)}
    for fpm, dfi in zip(['A', 'B'], [dfa, dfb]):
        with open(f'sig_cuts_{det_comb}{fpm}_lvl2.txt', 'w') as f:
            for iter in range(3):
                dfi['SIG'] = (dfi[det_comb] - dfi[det_comb].mean()) / dfi[det_comb].std()
                high_sig = dfi[dfi['SIG'] > sig_limit]

                for ind, row in high_sig.iterrows():
                    use[fpm][ind] = False
                    if row['SEQID'] not in running_list[fpm]:
                        running_list[fpm] = np.append(running_list[fpm], row['SEQID'])
                        outstring = f"{row['SEQID']}, {row['NAME']}, {row['EXPOSURE']}, {row['RA']}, {row['DEC']}, {row['PA']}, {iter}"
                        print(fpm, row['SRC'], row['EXPOSURE'], row['SIG'], outstring)
                        f.write(outstring+'\n')
                    dfi = dfi[dfi['SIG']<sig_limit].copy()

    # Now get rid of everything that you've already used for the next round of searching.
    dfa = dfa[use['A']].copy().reset_index(drop=True)
    dfb = dfb[use['B']].copy().reset_index(drop=True)

DET0123

A 0.899361065555895 18012.7878 6.826402400526829 70860002002, ABELL_3571, 18012.7878, 206.8134, -32.8622, 10.7378, 0
A 0.5605869175575012 50238.4182 6.126601499494454 60801007002, M87, 50238.4182, 187.663, 12.38, 357.9903, 0
A 0.5238178085651389 19420.8747 5.824800986003168 60801001002, M87, 19420.8747, 187.6469, 12.3844, 13.1069, 0
A 0.5336816804531651 45538.3816 5.820313974202668 60702051002, M87, 45538.3816, 187.6782, 12.4231, 77.7517, 0
A 0.5397917603160786 48565.0244 5.933548144534244 60702051001, M87, 48565.0244, 187.6868, 12.425, 94.9522, 0
A 0.3641866412909341 95305.5276 8.713214791349403 70801001002, ABELL_3266_POS1, 95305.5276, 67.808, -61.4552, 229.276, 1
A 0.4366143109111123 103010.8241 6.896454714191268 10801407002, Kepler, 103010.8241, 262.7093, -21.4198, 156.2111, 1
B 0.8477592977822191 18012.7878 7.154143069688652 70860002002, ABELL_3571, 18012.7878, 206.8134, -32.8622, 10.7378, 0
B 0.5293039366699982 50238.4182 6.316018582182646 60801007002, M87, 50238.4182, 1

In [7]:
print(len(running_list['A']))
print(len(running_list['B']))



10
11
